# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [3]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [4]:
from ollama import chat
message = [
    {"role":"user" , "content":"how are you"}
]
response = chat(model="deepseek-r1:1.5b",messages=message)
print(response['message']["content"])

<think>

</think>

Hello! I'm just a virtual assistant, so I don't have feelings, but I'm here and ready to help you with whatever you need. How can I assist you today? 😊


In [5]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):  
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [6]:
ed = Website("https://edwarddonner.com")
data =ed.get_contents()
print(data)

Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers a

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [7]:
link_system_prompt = (
    "You are provided with a list of links found on a webpage. "
    "You must decide which links would be most relevant to include in a brochure about the company, "
    "such as links to About, Company, Careers, or Jobs pages.\n\n"
    "You MUST respond with ONLY a valid JSON object and nothing else — no explanations, no thoughts, no text, no markdown formatting (no triple backticks).\n\n"
    "The JSON should look like this:\n\n"
    "{\n"
    "    \"links\": [\n"
    "        {\"type\": \"about page\", \"url\": \"https://full.url/goes/here/about\"},\n"
    "        {\"type\": \"careers page\", \"url\": \"https://another.full.url/careers\"}\n"
    "    ]\n"
    "}\n\n"
    "⚠️ IMPORTANT: Only output the JSON object. Do NOT include any commentary, explanation, thinking, or markdown code blocks."
)


In [8]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You must decide which links would be most relevant to include in a brochure about the company, such as links to About, Company, Careers, or Jobs pages.

You MUST respond with ONLY a valid JSON object and nothing else — no explanations, no thoughts, no text, no markdown formatting (no triple backticks).

The JSON should look like this:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}

⚠️ IMPORTANT: Only output the JSON object. Do NOT include any commentary, explanation, thinking, or markdown code blocks.


In [9]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [10]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/01/23/ll

In [51]:
from ollama import chat
import json
import re

def get_links(url):
    website = Website(url)
    response = chat(
        model="deepseek-r1:1.5b",
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
        ]
    )


    # Extract the content
    result = response.get("message", {}).get("content", "")
    
    if result:
        # Use regular expression to find the JSON part in the response
        match = re.search(r'\{.*\}', result, re.DOTALL)  # This regex matches the JSON part
        
        if match:
            json_content = match.group(0)  # Get the matched JSON block
            try:
                # Now try parsing the JSON content
                return json.loads(json_content)
            except json.JSONDecodeError:
                # If parsing fails, provide more information
                raise ValueError(f"Failed to parse content as JSON. The matched content is: {json_content}")
        else:
            raise ValueError("No valid JSON found in the response.")
    else:
        raise ValueError("The response content is empty or invalid.")


In [52]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/nari-labs/Dia-1.6B',
 '/sand-ai/MAGI-1',
 '/microsoft/bitnet-b1.58-2B-4T',
 '/ostris/Flex.2-preview',
 '/HiDream-ai/HiDream-I1-Full',
 '/models',
 '/spaces/nari-labs/Dia-1.6B',
 '/spaces/enzostvs/deepsite',
 '/spaces/InstantX/InstantCharacter',
 '/spaces/bytedance-research/UNO-FLUX',
 '/spaces/Kwai-Kolors/Kolors-Virtual-Try-On',
 '/spaces',
 '/datasets/nvidia/OpenMathReasoning',
 '/datasets/Anthropic/values-in-the-wild',
 '/datasets/zwhe99/DeepMath-103K',
 '/datasets/OpenGVLab/InternVL-Data',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '

In [53]:
get_links("https://huggingface.co")

{'links': [{'type': 'models', 'url': '//models'},
  {'type': 'spaces', 'url': '/spaces'},
  {'type': 'datasets', 'url': '/datasets'},
  {'type': 'posts', 'url': '/posts'},
  {'type': 'docs', 'url': '/docs'},
  {'type': 'enterprise', 'url': '/enterprise'},
  {'type': 'pricing#endpoints', 'url': '/pricing#endpoints'},
  {'type': 'Terms-of-Service', 'url': '/terms-of-service'},
  {'type': 'Privacy', 'url': '/privacy'},
  {'type': 'mail', 'url': 'mailto:press@huggingface.co'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [56]:
from urllib.parse import urljoin

def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    
    # Get the links
    links = get_links(url)
    print("Found links:", links)
    
    base_url = "https://huggingface.co"  # Base URL to prepend for relative links
    
    # Loop through each link and retrieve its contents
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        
        # Construct the full URL
        full_url = urljoin(base_url, link["url"])
        
        # Fetch the content of the full URL
        result += Website(full_url).get_contents()
        
    return result

# Test the function
print(get_all_details("https://huggingface.co"))


Found links: {'links': [{'type': 'about page', 'url': '/models'}, {'type': 'about page', 'url': '/datasets'}, {'type': 'about page', 'url': '/spaces'}, {'type': 'about page', 'url': '/nari-labs/Dia-1.6B'}, {'type': 'about page', 'url': '/sand-ai/MAGI-1'}, {'type': 'about page', 'url': '/Microsoft/bitnet-b1.58-2T'}, {'type': 'about page', 'url': '/ostris/Flex.2-preview'}, {'type': 'about page', 'url': '/HiDream-ai/HiDream-I1-Full'}, {'type': 'about page', 'url': '/spaces/nari-labs/Dia-1.6B'}, {'type': 'about page', 'url': '/spaces/enzostvs/deepsite'}, {'type': 'about page', 'url': '/spaces/Kwai-Kolors/Kolors-Virtual-Try-On'}, {'type': 'about page', 'url': '/spaces'}, {'type': 'about page', 'url': '/spaces/datasets/nvidia/OpenMathReasoning'}, {'type': 'about page', 'url': '/spaces/Anthropic/values-in-the-wild'}, {'type': 'about page', 'url': '/spaces/zwhe99/DeepMath-103K'}, {'type': 'about page', 'url': '/spaces/OpenGVLab/InternVL-Data'}, {'type': 'about page', 'url': '/spaces/nvidia/Ope

In [ ]:
print(get_all_details("https://huggingface.co"))

In [ ]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [ ]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

In [ ]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_brochure("HuggingFace", "https://huggingface.co")

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [58]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [59]:
stream_brochure("HuggingFace", "https://huggingface.co")

NameError: name 'openai' is not defined

In [60]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

NameError: name 'openai' is not defined

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>